## Meta matching v2.0
This jupyter notebook demonstrates you how to load and use multilayer meta-matching algorthm. In this demonstration, we performed multilayer meta-matching with 100 example subjects.

Package needed (and version this jupyter notebook tested):
* Numpy (1.19.5)
* Scipy (1.10.1)
* PyTorch (2.0.0)
* Scikit-learn (0.23.2)

### Step 0. Setup
Please modify the `path_repo` below to your repo position:

In [1]:
path_repo = '../' # '/home/the/deepGround/code/2002/Meta_matching_models/'

In [2]:
# initialization and random seed set

import os
import sys
import random
import scipy
import torch
import pickle
import sklearn
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

import warnings
warnings.filterwarnings("ignore")

### Step 1. Load data
Load the example data that we provided, it contains 
* Example input functional connectivity (FC) `x` with size of (100, 87571)
    * 100 is number of subjects
    * 87571 is flatten vector of 419 by 419 FC (419*418/2=87571)
* Example output phenotypes `y` with size of (100, 3)
    * 3 is number of phenotypes.

In [3]:
path_v20 = os.path.join(path_repo, 'v2.0')
model_v20_path = os.path.join(path_v20, 'models')
sys.path.append(os.path.join(path_repo, "utils"))
# check whether v2.0 model files exist and are up-to-date
from CBIG_model_pytorch import check_models_v20
check_models_v20(model_v20_path)
# load data
from CBIG_model_pytorch import demean_norm
path_data = os.path.join(path_repo, 'data')
npz = np.load(os.path.join(path_data, 'meta_matching_example_data.npz'))
x_input = npz['x']
y_input = npz['y']
x_input = demean_norm(x_input)

print(x_input.shape, y_input.shape)

(100, 87571) (100, 3)


### Step 2. Split data
Here, we also split 100 subjects to 80/20, where 80 for training, and 20 for test.

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_input, y_input, test_size=0.2, random_state=42)
n_subj_train, n_subj_test = x_train.shape[0], x_test.shape[0]
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(80, 87571) (20, 87571) (80, 3) (20, 3)


After data split, we z-normalize the label (y) of the training and testing set, using mean and std of the training set.

In [5]:
def z_norm(y_train, y_test):
    # subtract mean of y of the training set
    t_mu = np.nanmean(y_train, axis=0, keepdims=True)
    y_train = y_train - t_mu
    y_test = y_test - t_mu

    # divide std of y of the training set
    t_sigma = np.nanstd(y_train, axis=0)
    if y_train.ndim == 2:
        t_sigma_d = t_sigma[np.newaxis, :]
    else:
        t_sigma_d = t_sigma
        if t_sigma == 0:
            return y_train, y_test
    y_train = y_train / t_sigma_d
    y_test = y_test / t_sigma_d
    return y_train, y_test

y_train, y_test = z_norm(y_train, y_test)

### Step 3. Multilayer meta-matching models predict
Here we apply the DNN and RR models trained on extra-large source dataset (UK Biobank), large source dataset (ABCD) and medium source dataset (GSP, HBN and eNKI) to predict source phenotypes on `x_train` and `x_test`. We will get the predicted 458 source phenotypes on both 80 training subjects and 20 test subjects (`y_train_pred` and `y_test_pred`), and the corresponding phenotype names (`y_names`).

In [6]:
# Inference using all DNN + RR models from all source datasets
from CBIG_model_pytorch import multilayer_metamatching_infer
dataset_names = {'extra-large': 'UKBB', 'large': 'ABCD', 'medium': ['GSP', 'HBN', 'eNKI']}
y_train_pred, y_names = multilayer_metamatching_infer(x_train, y_train, model_v20_path, dataset_names)
y_test_pred, _ = multilayer_metamatching_infer(x_test, y_test, model_v20_path, dataset_names)

print(len(y_names))
print(y_train_pred.shape, '\n', y_train_pred)
print(y_test_pred.shape, '\n', y_test_pred)

458
(80, 458) 
 [[-0.10125838  0.03613428 -0.46239799 ...  0.39725083 -0.76892095
  -1.44623037]
 [ 0.10885195 -0.16720149  0.49574935 ...  3.06916403  2.31081115
   0.82206434]
 [ 0.10951973  0.2454676  -0.44824579 ... -0.80978381 -0.17417671
  -0.14402823]
 ...
 [ 0.13027874  0.01848663 -0.1462169  ...  0.85882294  0.5268896
   0.34999364]
 [-0.00888915 -0.21047002  0.36738572 ...  1.74289951  0.60727872
   0.73831846]
 [-0.08907671  0.17226079 -0.29705456 ... -0.22615359  0.2034359
   0.777701  ]]
(20, 458) 
 [[-0.06638002 -0.1506063   0.14014049 ...  1.52472606  1.45798438
   0.87642115]
 [-0.01658378 -0.20561379  0.38617522 ...  1.10521609 -0.05357165
   1.36535158]
 [ 0.04155716 -0.19629928  0.32053086 ...  1.45453133  0.99876183
   1.01717683]
 ...
 [ 0.05204854 -0.173134    0.32241014 ...  1.38678291  1.38245556
  -0.23924551]
 [-0.17038341 -0.13663808  0.15161921 ...  1.81635199  0.3067782
   1.71055638]
 [ 0.10234474  0.19847694 -0.43693691 ...  0.65697633  1.39920397
   0.85

### Step 4. Stacking
Perform stacking with `y_train_pred`, `y_test_pred`, `y_train`, where we use the prediction of 80 subjects `y_train_pred` (input) and real data `y_train` (output) to train the stacking model (you can either use all 67 source phenotypes for stacking, or select top K source phenotypes relevant to the target phenotype, like we mentioned in our paper; it turns out that these 2 ways achieves similar performances), then we applied the model to `y_test_pred` to get final prediction of 3 phenotypes on 20 subjects.

#### Hyperparameter Tuning 
In `stacking()` function, we set the range of `alpha` as `[0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20]`. You are weclomed to modify the range of `alpha` to get better performance on your own data.

In [7]:
from CBIG_model_pytorch import stacking
y_test_final=np.zeros((y_test_pred.shape[0], y_train.shape[1]))
for i in range(y_train.shape[1]):
    # For each test phenotype, perform stacking by developing a KRR model
    y_test_temp, _ = stacking(y_train_pred, y_test_pred, y_train[:,i].view(), [0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20])
    y_test_final[:,i] = y_test_temp.flatten()
print(y_test_final.shape, '\n', y_test_final)

(20, 3) 
 [[ 0.31547324  0.429776    0.24438339]
 [ 0.48541949  0.39782572  0.29578741]
 [ 0.2405772   0.3119502   0.49131335]
 [-0.54862517 -0.52383948 -0.4586865 ]
 [ 0.23397184  0.09786208 -0.12051541]
 [-0.00592956 -0.05972484 -0.58517721]
 [ 0.22268349  0.28420622  0.27605983]
 [-0.37521873 -0.34830537 -0.48129185]
 [-0.42836626 -0.48787542 -0.00697907]
 [ 0.17523012  0.03362288  0.13360624]
 [-0.36981469 -0.45021176 -0.36496257]
 [ 0.12878666  0.22143727  0.01007384]
 [ 0.32556518  0.29869739  0.20836058]
 [ 0.31937476  0.3363618   0.35522556]
 [-0.00916185 -0.05389288  0.22052705]
 [ 0.29450547  0.47023931  0.29891575]
 [-0.4655142  -0.27739325  0.13200606]
 [ 0.04682419  0.05586322 -0.64544138]
 [ 0.18472115  0.20659532  0.77945917]
 [-0.14501365 -0.17101814 -0.06838476]]


### Step 5. Evaluation
Evaluate the prediction performance. Note that we didn't reverse the z-normalization which previously applied on `y_train` and `y_test`. This is because the metric (Pearson correlation) would not be affected by z-normalization. If you are predicting the phenotypes for practical use, you are recommended to reverse the z-normalization

In [8]:
from scipy.stats.stats import pearsonr
corr = np.zeros((y_train.shape[1]))
for i in range(y_train.shape[1]):
    corr[i] = pearsonr(y_test_final[:, i], y_test[:, i])[0]
print(corr)

[0.39993445 0.33584784 0.36800187]


### Step 6. Haufe transform predictive network features (PNFs) computation
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each element of FC on the 80 training subjects, which helps identify how changes in FC are associated with increases or decreases in the phenotypic prediction. This process enables generating feature importance weights for each edge, indicating the contribution of each feature to the prediction. The final PNF is in shape of (87571, 3), where 87571 is number of 419 by 419 FC elements, and 3 is number of phenotypes. You can visualize PNF matrix in specific network order by calling our MATLAB function [CBIG_PlotCorrMatNetOrder](https://github.com/ThomasYeoLab/CBIG/blob/master/utilities/matlab/figure_utilities/PlotCorrMat/CBIG_PlotCorrMatNetOrder.m) (set res=400, netorder='Schaefer_Yeo17')

In [9]:
from CBIG_model_pytorch import covariance_rowwise

y_train_haufe, _ = stacking(y_train_pred, y_train_pred, y_train)
print(y_train_haufe.shape)
cov = covariance_rowwise(x_train, y_train_haufe)
print(cov, '\n', cov.shape)

(80, 3)
[[-1.54667507e-04 -1.32399773e-04 -1.31030976e-04]
 [-4.07118465e-05 -3.62762901e-05 -1.60762181e-04]
 [-1.74738927e-04 -1.50471013e-04 -3.19341539e-04]
 ...
 [ 1.21284993e-04  8.06889920e-05  1.50644705e-04]
 [ 1.65970726e-04  1.25041515e-04  1.95054270e-04]
 [ 1.15016185e-04  1.10306331e-04 -3.75667195e-05]] 
 (87571, 3)


### Step 7. Haufe transform predictive network features (PNFs) computation for training phenotypes
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each training phenotypes on the 80 training subjects, which helps identify which source phenotypes are more associated with target phenotype prediction. The final PNF is in shape of (458, 3), where 458 is the number of source phenotypic predictions, and 3 is number of phenotypes.

We also give the phenotype name of output in order, which is in format of \<Phenotype_name\>\_\<Dataset\>\_\<Model\>. The \<Model\> can be DNN/KRR base model trained on UK Biobank ('DNN'/'KRR'), one layer meta-matching ('1layer'), or two layer meta-matching ('2layer').

In [10]:
cov = covariance_rowwise(y_train_pred, y_train_haufe)
print(cov, '\n', cov.shape)
print(y_names) # source phenotype names in order

[[ 0.0054045   0.00075897 -0.01404931]
 [-0.0411503  -0.0377332   0.00257934]
 [ 0.10179323  0.09308122  0.02029504]
 ...
 [ 0.17937701  0.17730432  0.02461779]
 [ 0.09936957  0.10102418  0.02913543]
 [ 0.1538624   0.1626103   0.24924063]] 
 (458, 3)
['Alcohol_1_UKBB_DNN', 'Alcohol_2_UKBB_DNN', 'Alcohol_3_UKBB_DNN', 'Time_walk_UKBB_DNN', 'Time_drive_UKBB_DNN', 'Time_TV_UKBB_DNN', 'Sleep_UKBB_DNN', 'Age_edu_UKBB_DNN', 'Work_UKBB_DNN', 'Travel_UKBB_DNN', '#household_UKBB_DNN', 'Neuro_UKBB_DNN', 'Hearing_UKBB_DNN', 'Fluid_Int._UKBB_DNN', 'Match_UKBB_DNN', 'Sex_UKBB_DNN', 'Match-o_UKBB_DNN', 'Age_UKBB_DNN', 'Trail-o_C1_UKBB_DNN', 'Trail-o_C4_UKBB_DNN', 'Digit-o_C1_UKBB_DNN', 'Digit-o_C6_UKBB_DNN', 'Sex_G_C1_UKBB_DNN', 'Sex_G_C2_UKBB_DNN', 'Genetic_C1_UKBB_DNN', 'Cancer_C1_UKBB_DNN', 'Urine_C1_UKBB_DNN', 'Blood_C2_UKBB_DNN', 'Blood_C3_UKBB_DNN', 'Blood_C4_UKBB_DNN', 'Blood_C5_UKBB_DNN', 'Deprive_C1_UKBB_DNN', 'Dur_C1_UKBB_DNN', 'Dur_C2_UKBB_DNN', 'Dur_C4_UKBB_DNN', 'Trail_C1_UKBB_DNN', 'Tow